# Advanced Python: Building Scalable Applications

### Day 5: Serialization, Data Persistence and Web APIs


In [2]:
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.shutdown?

Docstring:
shutdown(flag)

Shut down the reading side of the socket (flag == SHUT_RD), the writing side
of the socket (flag == SHUT_WR), or both ends (flag == SHUT_RDWR).
Type:      builtin_function_or_method


In [3]:
pip install httpx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import httpx

res = httpx.get("http://python.org")
res

<Response [301 Moved Permanently]>

In [8]:
res.status_code, res.reason_phrase

(301, 'Moved Permanently')

In [10]:
print(res.is_redirect, res.is_success, res.is_error, res.is_client_error)

True False False False


In [12]:
for k, v in res.headers.items():
    print(f"{k} -> {v}")

connection -> close
content-length -> 0
server -> Varnish
retry-after -> 0
accept-ranges -> bytes
date -> Fri, 22 Nov 2024 05:07:12 GMT
via -> 1.1 varnish
x-served-by -> cache-maa10225-MAA
x-cache -> HIT
x-cache-hits -> 0
x-timer -> S1732252032.296288,VS0,VE0
location -> https://www.python.org/
strict-transport-security -> max-age=315360000; preload


In [13]:
res = httpx.get("https://www.python.org")
res

<Response [200 OK]>

In [14]:
print(res.is_redirect, res.is_success, res.is_error, res.is_client_error)

False True False False


In [15]:
for k, v in res.headers.items():
    print(f"{k} -> {v}")

connection -> keep-alive
content-length -> 12076
via -> 1.1 varnish, 1.1 varnish, 1.1 varnish
content-encoding -> gzip
x-frame-options -> SAMEORIGIN
content-type -> text/html; charset=utf-8
accept-ranges -> bytes
date -> Fri, 22 Nov 2024 05:13:08 GMT
age -> 1563
x-served-by -> cache-iad-kiad7000114-IAD, cache-iad-kiad7000114-IAD, cache-maa10240-MAA
x-cache -> MISS, MISS, HIT
x-cache-hits -> 0, 0, 3
x-timer -> S1732252388.050457,VS0,VE0
vary -> Cookie
strict-transport-security -> max-age=63072000; includeSubDomains; preload


In [16]:
res.text

'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <!-- Google tag (gtag.js) -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=G-TF35YF9CVH"></script>\n    <script>\n      window.dataLayer = window.dataLayer || [];\n      function gtag(){dataLayer.push(arguments);}\n      gtag(\'js\', new Date());\n      gtag(\'config\', \'G-TF35YF9CVH\');\n    </script>\n    <!-- Plausible.io analytics -->\n    <script defer data-domain="python.org" src="https://plausible.io/js/script.js"></script>\n\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js

In [17]:
res.content

b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <!-- Google tag (gtag.js) -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=G-TF35YF9CVH"></script>\n    <script>\n      window.dataLayer = window.dataLayer || [];\n      function gtag(){dataLayer.push(arguments);}\n      gtag(\'js\', new Date());\n      gtag(\'config\', \'G-TF35YF9CVH\');\n    </script>\n    <!-- Plausible.io analytics -->\n    <script defer data-domain="python.org" src="https://plausible.io/js/script.js"></script>\n\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.j

In [32]:
GEO_LOCATION_URL = "http://api.openweathermap.org/geo/1.0/direct"
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

GEO_LOCATION_PARAMS = {
    "q": None,
    "limit": "1",
    "appid": None
}

WEATHER_PARAMS = {
    "lat": None,
    "lon": None,
    "units": "metric",
    "appid": None
}

def get_api_key():
    with open(r"C:\Users\chandrashekar\open_weathermap_api_key.txt") as ins:
        return ins.readline()


def get_coords(location):
    GEO_LOCATION_PARAMS["q"] = location
    GEO_LOCATION_PARAMS["appid"] = get_api_key()
    res = httpx.get(GEO_LOCATION_URL, params=GEO_LOCATION_PARAMS)
    if res.is_success and "/json" in res.headers["content-type"]:
        return res.json()[0]["lat"], res.json()[0]["lon"]
    else:
        raise ValueError(f"Request failed: {res}")
    
def get_weather(city):
    from time import sleep
    api_key = get_api_key()
    lat, lon = get_coords(city)
    WEATHER_PARAMS["lat"] = lat
    WEATHER_PARAMS["lon"] = lon
    WEATHER_PARAMS["appid"] = get_api_key()
    sleep(1) # Might be needed for the free-tier to avoid bans!
    res = httpx.get(WEATHER_URL, params=WEATHER_PARAMS)
    if res.is_success and "/json" in res.headers["content-type"]:
        return res.json()["main"]["temp"], res.json()["main"]["humidity"]
    else:
        raise ValueError(f"Request failed: {res}")


t, h = get_weather("Bengaluru")
print(f"Delhi records {t} degrees celsius and humidity of {h} percent")


Delhi records 25.65 degrees celsius and humidity of 61 percent


In [ ]:
import httpx

with open(r"C:\Users\chandrashekar\open_weathermap_api_key.txt") as ins:
    API_KEY = ins.readline()



'65bda05da4f075dc2bd5ddc75240316e'

In [19]:
GEO_LOCATION_URL = "http://api.openweathermap.org/geo/1.0/direct"
GEO_LOCATION_PARAMS = {
    "q": None,
    "limit": "1",
    "appid": API_KEY
}

GEO_LOCATION_PARAMS["q"] = "Chennai"

res = httpx.get(GEO_LOCATION_URL, params=GEO_LOCATION_PARAMS)
res

<Response [200 OK]>

In [24]:
res.headers["content-type"]
lat, lon = res.json()[0]["lat"], res.json()[0]["lon"]
lat, lon

(13.0836939, 80.270186)

In [42]:
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
WEATHER_PARAMS = {
    "lat": lat,
    "lon": lon,
    "units": "metric",
    "mode": "html",
    "appid": API_KEY
}

res = httpx.get(WEATHER_URL, params=WEATHER_PARAMS)
res

<Response [200 OK]>

In [43]:
res.headers["content-type"]

'text/html; charset=utf-8'

In [44]:
res.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="utf-8">\n  <meta name="keywords" content="weather, world, openweathermap, weather, layer" />\n  <meta name="description" content="A layer with current weather conditions in cities for world wide" />\n  <meta name="domain" content="openweathermap.org" />\n  <meta http-equiv="pragma" content="no-cache" />\n  <meta http-equiv="Expires" content="-1" />\n</head>\n<body>\n  <div style="font-size: medium; font-weight: bold; margin-bottom: 0px;">Park Town</div>\n  <div style="float: left; width: 130px;">\n    <div style="display: block; clear: left;">\n      <div style="float: left;" title="Titel">\n        <img height="45" width="45" style="border: medium none; width: 45px; height: 45px; background: url(&quot;http://openweathermap.org/img/w/50d.png&quot;) repeat scroll 0% 0% transparent;" alt="title" src="http://openweathermap.org/images/transparent.png"/>\n      </div>\n      <div style="float: left;">\n        <div style="display:

AttributeError: 'function' object has no attribute 'HTML'

In [29]:
res.json()["main"]["temp"]

29.16

### Common HTTP verbs (methods) in use for API:
  1. GET /resource    - Fetch a resource (SELECT in SQL)
  2. POST /resource   - Create a resource (INSERT in SQL) 
  3. PUT /resource    - Replace a resource (REPLACE in SQL)
  4. PATCH /resource  - Update a resource (UPDATE in SQL)
  5. DELETE /resource - Delete a resource (DELETE in SQL)

### In pure RESTful idioms: resources are of two types:
    - Collection resource
      - GET /employees -> should return a "list" of employees
      - POST /employees -> add a new record into the collection
  
    - Singular resource
      - GET /employees/employee_id -> Should return a single record representing the resource
      - PUT /employees/employee_id -> Replace the employee for the given employee_id
      - PATCH /employees/employee_id
      - DELETE /employees/employee_id
  
  
   

In [37]:
httpx.get?

Signature:
httpx.get(
    url: 'URL | str',
    *,
    params: 'QueryParamTypes | None' = None,
    headers: 'HeaderTypes | None' = None,
    cookies: 'CookieTypes | None' = None,
    auth: 'AuthTypes | None' = None,
    proxy: 'ProxyTypes | None' = None,
    proxies: 'ProxiesTypes | None' = None,
    follow_redirects: 'bool' = False,
    cert: 'CertTypes | None' = None,
    verify: 'VerifyTypes' = True,
    timeout: 'TimeoutTypes' = Timeout(timeout=5.0),
    trust_env: 'bool' = True,
) -> 'Response'
Docstring:
Sends a `GET` request.

**Parameters**: See `httpx.request`.

Note that the `data`, `files`, `json` and `content` parameters are not available
on this function, as `GET` requests should not include a request body.
File:      c:\python310\lib\site-packages\httpx\_api.py
Type:      function


In [50]:
httpx.get("http://python.org/", follow_redirects=True)

<Response [200 OK]>

In [51]:
httpx.get("https://expired.badssl.com/")

ConnectError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)

In [52]:
httpx.get("https://expired.badssl.com/", verify=False)

<Response [200 OK]>

In [53]:
httpx.post?

Signature:
httpx.post(
    url: 'URL | str',
    *,
    content: 'RequestContent | None' = None,
    data: 'RequestData | None' = None,
    files: 'RequestFiles | None' = None,
    json: 'typing.Any | None' = None,
    params: 'QueryParamTypes | None' = None,
    headers: 'HeaderTypes | None' = None,
    cookies: 'CookieTypes | None' = None,
    auth: 'AuthTypes | None' = None,
    proxy: 'ProxyTypes | None' = None,
    proxies: 'ProxiesTypes | None' = None,
    follow_redirects: 'bool' = False,
    cert: 'CertTypes | None' = None,
    verify: 'VerifyTypes' = True,
    timeout: 'TimeoutTypes' = Timeout(timeout=5.0),
    trust_env: 'bool' = True,
) -> 'Response'
Docstring:
Sends a `POST` request.

**Parameters**: See `httpx.request`.
File:      c:\python310\lib\site-packages\httpx\_api.py
Type:      function


In [54]:
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
WEATHER_PARAMS = {
    "lat": lat,
    "lon": lon,
    "units": "metric",
    #"mode": "html",
    "appid": API_KEY
}

res = httpx.get(WEATHER_URL, params=WEATHER_PARAMS)
res

<Response [200 OK]>

In [55]:
res.json()

{'coord': {'lon': 80.2702, 'lat': 13.0837},
 'weather': [{'id': 721,
   'main': 'Haze',
   'description': 'haze',
   'icon': '50d'}],
 'base': 'stations',
 'main': {'temp': 29.91,
  'feels_like': 34.59,
  'temp_min': 29.46,
  'temp_max': 30.5,
  'pressure': 1011,
  'humidity': 69,
  'sea_level': 1011,
  'grnd_level': 1010},
 'visibility': 4000,
 'wind': {'speed': 4.63, 'deg': 30},
 'clouds': {'all': 40},
 'dt': 1732259441,
 'sys': {'type': 2,
  'id': 2093220,
  'country': 'IN',
  'sunrise': 1732236072,
  'sunset': 1732277358},
 'timezone': 19800,
 'id': 1465730,
 'name': 'Park Town',
 'cod': 200}

In [59]:
import json

a = {"name": "John", "visited": ("Chennai", "Bengaluru", "Noida"), "active": False, "logged_in": None}

s = json.dumps(a)
print(s)
d = json.loads(s)
print(d)

{"name": "John", "visited": ["Chennai", "Bengaluru", "Noida"], "active": false, "logged_in": null}
{'name': 'John', 'visited': ['Chennai', 'Bengaluru', 'Noida'], 'active': False, 'logged_in': None}


In [ ]:
import json
a = {"name": "John", "visited": ("Chennai", "Bengaluru", "Noida"), "active": False, "logged_in": None}

with open("testfile.json", "w") as outs:
    json.dump(a, outs)

# Duck typing

In [61]:
import json
a = {"name": "John", "visited": ("Chennai", "Bengaluru", "Noida"), "active": False, "logged_in": None}

with open("testfile.json", "r") as ins:
    d = json.load(ins)

print(d)
# Duck typing

{'name': 'John', 'visited': ['Chennai', 'Bengaluru', 'Noida'], 'active': False, 'logged_in': None}


In [63]:
import toml

with open("a.toml", "w") as outs:
    tomllib.dump(a, outs)

ModuleNotFoundError: No module named 'toml'

In [66]:
# pip install msgpack
import msgpack

m = msgpack.dumps(a)
j = json.dumps(a)
print(len(m), len(j))

63 98


In [71]:
import pickle

print(a)
d = pickle.dumps(a)
print(d)

with open("data.dat", "wb") as outs:
    pickle.dump(a, outs)

{'name': 'John', 'visited': ('Chennai', 'Bengaluru', 'Noida'), 'active': False, 'logged_in': None}
b'\x80\x04\x95T\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x04name\x94\x8c\x04John\x94\x8c\x07visited\x94\x8c\x07Chennai\x94\x8c\tBengaluru\x94\x8c\x05Noida\x94\x87\x94\x8c\x06active\x94\x89\x8c\tlogged_in\x94Nu.'


In [72]:
with open("data.dat", "rb") as ins:
    d = pickle.load(ins)
print(d)

{'name': 'John', 'visited': ('Chennai', 'Bengaluru', 'Noida'), 'active': False, 'logged_in': None}


In [74]:
pip install toml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [75]:
import toml

In [76]:
with open("a.toml", "w") as outs:
    toml.dump(a, outs)

In [77]:
with open("a.toml") as ins:
    d = toml.load(ins)
print(d)

{'name': 'John', 'visited': ['Chennai', 'Bengaluru', 'Noida'], 'active': False}


In [97]:
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
WEATHER_PARAMS = {
    "lat": lat,
    "lon": lon,
    "units": "metric",
    "mode": "xml",
    "appid": API_KEY
}

res = httpx.get(WEATHER_URL, params=WEATHER_PARAMS)
res

<Response [200 OK]>

In [98]:
res.headers["content-type"]

'application/xml; charset=utf-8'

In [80]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [81]:
# lxml -> Implements ElementTree specifications

# For parsing / processing XML documents
#import xml.etree.ElementTree as et
import lxml.etree as et

# For parsing / processing HTML documents
import lxml.html as et



In [82]:
et.parse?

Signature: et.parse(filename_or_url, parser=None, base_url=None, **kw)
Docstring:
Parse a filename, URL, or file-like object into an HTML document
tree.  Note: this returns a tree, not an element.  Use
``parse(...).getroot()`` to get the document root.

You can override the base URL with the ``base_url`` keyword.  This
is most useful when parsing from a file-like object.
File:      c:\python310\lib\site-packages\lxml\html\__init__.py
Type:      function


In [90]:
print(res.text)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="keywords" content="weather, world, openweathermap, weather, layer" />
  <meta name="description" content="A layer with current weather conditions in cities for world wide" />
  <meta name="domain" content="openweathermap.org" />
  <meta http-equiv="pragma" content="no-cache" />
  <meta http-equiv="Expires" content="-1" />
</head>
<body>
  <div style="font-size: medium; font-weight: bold; margin-bottom: 0px;">Park Town</div>
  <div style="float: left; width: 130px;">
    <div style="display: block; clear: left;">
      <div style="float: left;" title="Titel">
        <img height="45" width="45" style="border: medium none; width: 45px; height: 45px; background: url(&quot;http://openweathermap.org/img/w/50d.png&quot;) repeat scroll 0% 0% transparent;" alt="title" src="http://openweathermap.org/images/transparent.png"/>
      </div>
      <div style="float: left;">
        <div style="display: block; clear: left

In [92]:
res = httpx.get("https://python.org", follow_redirects=True)
res

<Response [200 OK]>

In [93]:
tree = et.fromstring(res.text)

tree[1][0]

<!-- Google tag (gtag.js) -->

In [95]:
tree.xpath(".//a[@href]/@href")

['#content',
 '#python-network',
 '/',
 'https://www.python.org/psf/',
 'https://docs.python.org',
 'https://pypi.org/',
 '/jobs/',
 '/community/',
 '#top',
 '/',
 'https://psfmember.org/civicrm/contribute/transact?reset=1&id=2',
 '#site-map',
 '#',
 'javascript:;',
 'javascript:;',
 'javascript:;',
 '#',
 'https://www.linkedin.com/company/python-software-foundation/',
 'https://fosstodon.org/@ThePSF',
 '/community/irc/',
 'https://twitter.com/ThePSF',
 '/about/',
 '/about/apps/',
 '/about/quotes/',
 '/about/gettingstarted/',
 '/about/help/',
 'http://brochure.getpython.info/',
 '/downloads/',
 '/downloads/',
 '/downloads/source/',
 '/downloads/windows/',
 '/downloads/macos/',
 '/download/other/',
 'https://docs.python.org/3/license.html',
 '/download/alternatives',
 '/doc/',
 '/doc/',
 '/doc/av',
 'https://wiki.python.org/moin/BeginnersGuide',
 'https://devguide.python.org/',
 'https://docs.python.org/faq/',
 'http://wiki.python.org/moin/Languages',
 'https://peps.python.org',
 'https

In [96]:
tree.xpath(".//img[@src]/@src")

['/static/img/python-logo.png']

In [106]:
import lxml.etree as et
tree = et.fromstring(res.content)
tree.xpath(".//temperature/@value"), tree.xpath(".//humidity/@value")

(['30.07'], ['70'])

In [102]:
print(res.text)

<?xml version="1.0" encoding="UTF-8"?>
<current><city id="1465730" name="Park Town"><coord lon="80.2702" lat="13.0837"></coord><country>IN</country><timezone>19800</timezone><sun rise="2024-11-22T00:41:12" set="2024-11-22T12:09:18"></sun></city><temperature value="30.07" min="29.47" max="30.61" unit="celsius"></temperature><feels_like value="35.21" unit="celsius"></feels_like><humidity value="70" unit="%"></humidity><pressure value="1010" unit="hPa"></pressure><wind><speed value="4.63" unit="m/s" name="Gentle Breeze"></speed><gusts></gusts><direction value="40" code="NE" name="NorthEast"></direction></wind><clouds value="40" name="scattered clouds"></clouds><visibility value="4000"></visibility><precipitation mode="no"></precipitation><weather number="721" value="haze" icon="50d"></weather><lastupdate value="2024-11-22T08:05:28"></lastupdate></current>
